In [3]:
import numpy as np
import tensorflow as tf
import tensorflow_compression as tfc
import tensorflow_probability as tfp
import matplotlib.pyplot as plt

from time import time
from tqdm import tqdm

import os

2023-03-04 13:39:39.808055: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-04 13:39:45.632101: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-04 13:39:47.031019: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-04 13:39:47.031037: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [4]:
class Decoder(tf.keras.layers.Layer):
    """Encoder network for the VAE."""
    
    def __init__(self, N):
        """Initializes the encoder."""
        
        super(Decoder, self).__init__()
        self.N      = N
        self.conv2  = tf.keras.layers.Conv2DTranspose(self.N, 5, strides=2)
        self.conv1  = tf.keras.layers.Conv2DTranspose(self.N, 5, strides=2)
        self.conv3  = tf.keras.layers.Conv2DTranspose(self.N, 5, strides=2)
        self.conv4  = tf.keras.layers.Conv2DTranspose(3, 5, strides=2)
        self.gdn1   = tfc.layers.GDN(inverse=True)
        self.gdn2   = tfc.layers.GDN(inverse=True)
        self.gdn3   = tfc.layers.GDN(inverse=True)
    
    def call(self, inputs):
        """Forward pass of the decoder."""
        x = self.conv1(inputs)
        x = self.gdn1(x)
        x = self.conv2(x)
        x = self.gdn2(x)
        x = self.conv3(x)
        x = self.gdn3(x)
        z = self.conv4(x)
        return z

In [5]:
def indexes(i):
    return i

def get_indexed_emodel(num_scales):
    return tfc.LocationScaleIndexedEntropyModel(
        prior_fn=tfc.NoisyNormal,
        num_scales=num_scales,
        scale_fn = indexes,
        coding_rank=1,
     )

def get_batched_emodel(batch_shape=()):
    return tfc.ContinuousBatchedEntropyModel(
        prior=tfc.distributions.NoisyDeepFactorized(batch_shape=batch_shape),
        coding_rank=1
    )


In [19]:
# bemodel = get_batched_emodel((19, 19))
iemodel = get_indexed_emodel(19)

# y_tilde, rate_i = bemodel
y     = tf.random.uniform((2,2))
sigma = 0.5
y_tilde, rate_i = iemodel(
                        y, 
                        sigma, 
                        training=True,
                        )

print(y_tilde)
print(rate_i)


tf.Tensor(
[[ 0.6269213  -0.28662133]
 [ 1.0749947   0.8063948 ]], shape=(2, 2), dtype=float32)
tf.Tensor([2.0862703 4.9217143], shape=(2,), dtype=float32)
